In [1]:
import os
import gc
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import sys
sys.path.append( '../util' )
import util as util

/var/folders/1s/tyt3m2l12dz6s028_blqch7r0000gn/T/ipykernel_86843/1971013312.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def create_multiclass_classification_model(input_shape, num_classes):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu')(inputs)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x) 
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
train, test = util.import_dataset(7,"dnn")
y_train = train[util.y_column]
y_test = test[util.y_column]

X_train = train.drop(util.y_column, axis=1)
X_test = test.drop(util.y_column, axis=1)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

del train,test,y_train,y_test
num_classes = len(label_encoder.classes_)
print(" Number of classes is:" )
print(num_classes)
print("")
model = create_multiclass_classification_model(len(util.X_columns),num_classes)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#https://keras.io/api/models/model_training_apis/
history = model.fit(x=X_train, y=y_train_encoded,
                    validation_split=0.2, epochs=100, 
                    batch_size=256, callbacks=[early_stopping])


test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=2)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')
del X_train,X_test,y_train_encoded,y_test_encoded

Importing datasets.....
Processing training and test data....
Finished processing training and test data
Mapping labels to seven classes.....
Data import and processing complete....
 Number of classes is:
8

Epoch 1/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 206s 2ms/step - accuracy: 0.9232 - loss: 0.1789 - val_accuracy: 0.9833 - val_loss: 0.0457
Epoch 2/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 209s 2ms/step - accuracy: 0.9593 - loss: 0.0949 - val_accuracy: 0.9896 - val_loss: 0.0290
Epoch 3/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 1154s 10ms/step - accuracy: 0.9675 - loss: 0.0755 - val_accuracy: 0.9897 - val_loss: 0.0272
Epoch 4/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 181s 2ms/step - accuracy: 0.9738 - loss: 0.0634 - val_accuracy: 0.9892 - val_loss: 0.0283
Epoch 5/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 135s 1ms/step - accuracy: 0.9739 - loss: 0.0608 - val_accuracy: 0.9902 - val_loss: 0.0262
Epoch 6/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 183s 2ms/step - accuracy: 0.9758 - loss: 0.0572 - val_accu

In [ ]:
import matplotlib.pyplot as plt
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')
# Plot accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy for 7 classes')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
model.save('./my_saved_model')